## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
directory = '/content/drive/MyDrive/DACON_신용카드 사기 거래 탐지 AI 경진대회'
train_df = pd.read_csv(f'{directory}/data/train.csv') # Train
train_df = train_df.drop(columns=['ID'])
#display(train_df.head())

val_df = pd.read_csv(f'{directory}/data/val.csv') # Validation
val_df = val_df.drop(columns=['ID'])
#display(val_df.head())

test_df = pd.read_csv(f'{directory}/data/test.csv') # test
#display(test_df.head())

## 하이퍼파라미터

In [ ]:
EPOCHS = 400
LR = 1e-2
BS = 16384 # 한번의 학습시 대략 7번으로 나누어서 들어감
SEED = 41

In [ ]:
len(train_df)/16384

6.9483642578125

## 시드고정

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED) # Seed 고정

## 데이터셋 생성

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values

    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)

    def __len__(self):
        return len(self.df)

In [ ]:
train_dataset = MyDataset(df=train_df, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)#, num_workers=6)

val_dataset = MyDataset(df = val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)#, num_workers=6)
## 우리의 경우 num_workers = 1로 변경

## 1D AutoEncoder

* 차원확장 유추(gan)
Generative Adversial Network(GAN)은 훈련데이터의 확률분포를 학습하는 대표적인 생산적 모델 (Genertive Model)로 여러 분야에 활용되고 있다. 최근 모바일 장치를 이용한 포지셔닝 데이터의 대량수집이 가능해지면서 GAN을 활용해 위치데이터(위도,경도)를 생산하는 연구가 있었다. 하지만, 훈련 데이터가 위치데이터와 같이 복잡한 분포를 가지는 저차원 데이터인 경우 GAN의 학습이 불안정해진다는 문제점이 있다. 본 논문은 기본적인 Auto Encoder(AE)를 이용해 위치데이터의 차원을 확장시키는 방법을 제시한다. 실험을 통해, 해당 방법으로 차원이 늘어난 데이터를 GAN에 학습시킨다면 학습 안정에 효과가 있고, 의미있는 학습이 가능하다는 것을 확인하였다.

In [ ]:
## 배치 정규화란 : 학습 과정에서 각 배치 단위 별로 데이터가 다양한 분포를 가지더라도 각 배치별로 평균과 분산을 이용해 정규화하는 것
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.Encoder = nn.Sequential(
            nn.Linear(30,16),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),# 일반렐루와 다르게 임계치 보다 작으면 0이 아닌 0.01을 곱함
            nn.Linear(16,8),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),
        )
        self.Decoder = nn.Sequential(
            nn.Linear(8,16),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
            nn.Linear(16,30),
        )

    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

## Train (학습)

In [ ]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)

    def fit(self, ):
        self.model.to(self.device)
        best_score = 0
        for epoch in range(EPOCHS): #  몇번 학습시킬것인가
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device) # 인풋데이터
                self.optimizer.zero_grad()

                _x = self.model(x) # 아웃풋데이터
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score < score:
                best_score = score
                torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)

    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6) # 기본값은 1, 1e-8임
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)

                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist() # 인풋과 아웃풋의 코사인 유사도를 계산함
                batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

In [ ]:
# 코사인 유사도 예시
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)
output

tensor([ 0.0389,  0.0564, -0.0141, -0.0582, -0.0576, -0.0354, -0.0445,  0.0563,
        -0.0292,  0.1327,  0.0099,  0.0009,  0.0353,  0.0475, -0.1926,  0.0227,
         0.0359, -0.1252,  0.1518, -0.0207, -0.0415,  0.0630, -0.0226,  0.1588,
         0.1386, -0.0470, -0.0066, -0.0038,  0.0211,  0.0135,  0.0149,  0.0828,
         0.1035,  0.0355,  0.1377, -0.0357, -0.0584,  0.0028, -0.1251,  0.0115,
         0.0532,  0.0416, -0.0367, -0.0908, -0.0151,  0.0105,  0.0899,  0.2239,
         0.0566, -0.0742, -0.1069, -0.0435, -0.0076, -0.1017,  0.1429,  0.1977,
         0.1413, -0.0622, -0.1238,  0.0459,  0.0180, -0.0466, -0.0404,  0.0603,
        -0.1269,  0.0574,  0.0205, -0.0396,  0.1012, -0.0115,  0.0382,  0.0323,
         0.0757,  0.0109, -0.0725,  0.0365,  0.0082,  0.0654,  0.0313, -0.1474,
         0.0575, -0.0471,  0.0572,  0.1803,  0.0348,  0.0706, -0.0663, -0.1183,
        -0.1083, -0.0072, -0.0803,  0.0861, -0.0229, -0.0129, -0.0106,  0.0460,
        -0.0236, -0.0057, -0.0613,  0.07

## 모델 학습

In [ ]:
model = nn.DataParallel(AutoEncoder())
model.eval()
# eval(), train() : eval 모드시 평과 가정에서 사용하지 않을 레이어들의 전원을 끈다
# 사용하지 않는 레이어들의 예시 : Dropout, BatchNorm 등등

# optimizer : 모델학습시 예측결과의 차이를를 잘 줄일수 있게 해주는 역할
# scheduler : 가중치의 보폭인 학습률을 조정하는 역할
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Epoch : [0] Train loss : [0.6721689360482352] Val Score : [0.0010529271374420891])
Epoch : [1] Train loss : [0.6229286023548671] Val Score : [0.0010529271374420891])
Epoch : [2] Train loss : [0.5939646192959377] Val Score : [0.0010529271374420891])
Epoch : [3] Train loss : [0.5610476051058088] Val Score : [0.0010529271374420891])
Epoch : [4] Train loss : [0.5299920269421169] Val Score : [0.0010529271374420891])
Epoch : [5] Train loss : [0.5058594644069672] Val Score : [0.0016862816792256323])
Epoch : [6] Train loss : [0.48562902212142944] Val Score : [0.007002887533176871])
Epoch : [7] Train loss : [0.4679757910115378] Val Score : [0.01442751911500396])
Epoch : [8] Train loss : [0.4531616270542145] Val Score : [0.016274973944497156])
Epoch : [9] Train loss : [0.44054137808935984] Val Score : [0.014872907836955175])
Epoch : [10] Train loss : [0.4302045702934265] Val Score : [0.015523140155543836])
Epoch : [11] Train loss : [0.4216798799378531] Val Score : [0.01716203221341375])
Epoch : 

## 추론

In [ ]:
model = AutoEncoder()
model.load_state_dict(torch.load('./best_model.pth'))
model = nn.DataParallel(model)
model.eval()

In [ ]:
#test_df = pd.read_csv('./test.csv')
test_df = test_df.drop(columns=['ID'])

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values

    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)

    def __len__(self):
        return len(self.df)

In [ ]:
test_dataset = MyDataset(test_df, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=6)

In [ ]:
def prediction(model, thr, test_loader, device):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)

            _x = model(x)

            diff = cos(x, _x).cpu().tolist()
            batch_pred = np.where(np.array(diff)<thr, 1,0).tolist() # thr = 0.95
            pred += batch_pred
    return pred

def prediction_raw(model, thr, test_loader, device):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)
            print(x.shape)
            print(x)

            _x = model(x)

            diff = cos(x, _x).cpu().tolist()
            print('diff')
            print(diff.shape)
            #batch_pred = np.where(np.array(diff)<thr, 1,0).tolist() # thr = 0.95
            pred += diff
    return pred

In [ ]:
print(
len(preds),
len(list(filter(lambda x : x == 1, preds))),
len(list(filter(lambda x : x == 0, preds))),
len(list(filter(lambda x : x == 1, preds)))/len(list(filter(lambda x : x == 0, preds))))

142503 335 142168 0.0023563671149625797


In [ ]:
import random
print(random.sample(preds_raw, 3))

[0.9989621043205261, 0.998663067817688, 0.9990600347518921]


In [ ]:
submit = pd.read_csv(f'{directory}/data/sample_submission.csv')
submit['Class'] = preds
submit.to_csv(f'{directory}/submit/autoencoder_down_16_8_4_2_by_sh.csv', index=False)

In [ ]:
#자카르트 유사도 예시
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

list1 = ["삼성전자", "테슬라", "LG전자", "카카오", "펄어비스"]
list2 = ["삼성전자", "카카오", "넷마블", "현대자동차", "셀트리온"]

print('jaccard_similarity : ', jaccard_similarity(list1, list2))
출처: https://needjarvis.tistory.com/705 [자비스가 필요해:티스토리]